## ArXiv 数据加载与预处理

核心逻辑：

1. 处理 abstract + title: 使用 sentence-transformers 实时生成 384维 语义向量。

2. 处理 category: 简单的 Label Encoding。

3. 重点：将 384维 向量拆解为 384 个 DenseFeat 喂给模型。

In [ ]:
# ==========================================
# 📄 ArXiv 数据加载器 (含文本向量化)
# ==========================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, DenseFeat

# 尝试导入 sentence_transformers，如果没有则用随机向量代替（防止报错）
try:
    from sentence_transformers import SentenceTransformer
    HAS_BERT = True
except ImportError:
    HAS_BERT = False
    print("⚠️ 未安装 sentence-transformers，将使用随机向量模拟数据。")

def load_arxiv_data(csv_path):
    print(f"Loading ArXiv data from: {csv_path} ...")
    data = pd.read_csv(csv_path)
    
    # 1. 基础特征编码
    # Item ID (String -> Int)
    lbe_id = LabelEncoder()
    data['item_id_idx'] = lbe_id.fit_transform(data['item_id'])
    
    # Category (cs.CV -> 0, cs.RO -> 1)
    lbe_cat = LabelEncoder()
    data['category_idx'] = lbe_cat.fit_transform(data['category'])
    
    # 2. 文本向量化 (Text Embedding)
    # 我们使用 Title + Abstract 来生成语义向量
    EMBEDDING_DIM = 384 # MiniLM 的标准维度
    
    if HAS_BERT:
        print("⏳ 正在调用 BERT 模型生成摘要向量 (这可能需要几秒钟)...")
        model = SentenceTransformer('all-MiniLM-L6-v2')
        # 拼接 Title 和 Abstract 增加语义丰富度
        sentences = (data['title'] + ". " + data['abstract']).tolist()
        text_vectors = model.encode(sentences) # Shape: (N, 384)
    else:
        # 模拟数据 (仅调试用)
        text_vectors = np.random.random((len(data), EMBEDDING_DIM)).astype(np.float32)

    # 3. 定义特征配置
    # Sparse 部分: ID 和 Category
    feature_columns = [
        SparseFeat(name='item_id_idx', vocabulary_size=data['item_id_idx'].max() + 1, embedding_dim=16),
        SparseFeat(name='category_idx', vocabulary_size=data['category_idx'].max() + 1, embedding_dim=16)
    ]
    
    # Dense 部分: 把 384 维向量拆成 384 个 DenseFeat
    # 这是让 DeepFM 处理 BERT 向量的标准做法
    for i in range(EMBEDDING_DIM):
        feature_columns.append(DenseFeat(name=f'v_{i}', dimension=1))
        
    dnn_feature_columns = feature_columns
    linear_feature_columns = feature_columns

    # 4. 组装输入字典
    model_input = {
        'item_id_idx': data['item_id_idx'].values,
        'category_idx': data['category_idx'].values
    }
    
    # 把向量拆开塞进字典 (v_0, v_1 ... v_383)
    for i in range(EMBEDDING_DIM):
        model_input[f'v_{i}'] = text_vectors[:, i]
        
    # Label
    target = data['label'].values
    
    return model_input, linear_feature_columns, dnn_feature_columns, target

# --- 测试运行 ---
# arxiv_input, arxiv_linear, arxiv_dnn, arxiv_y = load_arxiv_data('arxiv_data.csv')
# print("ArXiv Input Keys Count:", len(arxiv_input)) # 应该是 2 + 384 = 386 个 key

## ArXiv 模型训练 (Training)

In [ ]:
import torch
from deepctr_torch.models import DeepFM
from deepctr_torch.callbacks import EarlyStopping

# 1. 准备数据
# 确保你已经运行了 Cell 2 定义了 load_arxiv_data
arxiv_csv_path = '../data/arxiv/train_arxiv_Researcher_LLM.csv'
arxiv_vec_path = 'arxiv_vectors.npy'
input_dict_arxiv, linear_cols_arxiv, dnn_cols_arxiv, target_arxiv = load_arxiv_data(arxiv_csv_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 2. 初始化 DeepFM 模型
# ArXiv 的特征维度很大 (384维)，建议把 DNN 层设宽一点
model_arxiv = DeepFM(linear_feature_columns=linear_cols_arxiv, 
                     dnn_feature_columns=dnn_cols_arxiv, 
                     task='binary', 
                     dnn_hidden_units=(256, 128), 
                     device=device)

# 3. 编译与训练
model_arxiv.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy', 'auc'])

history_arxiv = model_arxiv.fit(input_dict_arxiv, target_arxiv, 
                                batch_size=32, 
                                epochs=10, 
                                verbose=2, 
                                validation_split=0.2)

print("🎉 ArXiv 模型训练完成！")

## 模型保存 (Model Saving)

In [ ]:
# ==========================================
# 💾 模型权重保存
# ==========================================
import torch
import os

# 1. 定义保存路径 (建议放在当前工作目录)
arxiv_model_path = 'deepfm_arxiv_weights.pth'

# 2. 保存 ArXiv 模型
if 'model_arxiv' in locals():
    torch.save(model_arxiv.state_dict(), arxiv_model_path)
    print(f"✅ ArXiv 模型权重已保存至: {arxiv_model_path}")
else:
    print("⚠️ 未检测到 model_arxiv，跳过保存。")

print("\n提示: 在 ModelScope 左侧文件列表中应该能看到这两个 .pth 文件了。")

## 加载权重并启动 API 服务 (Loading & Serving)

In [ ]:
# ==========================================
# 🚀 模型加载与 API 服务启动
# ==========================================
import torch
import pandas as pd
import numpy as np
from deepctr_torch.models import DeepFM
from flask import Flask, request, jsonify
import threading

# 1. 重新定义一些基础配置 (防止重启内核后变量丢失)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
ARXIV_CSV = '../data/arxiv/train_arxiv_Researcher_LLM.csv'  # 确保文件名对
ARXIV_VEC = 'arxiv_vectors.npy'

# 2. 定义加载函数 (复用之前的逻辑，只为了获取特征列配置来初始化模型)
#    注意：这里我们需要 import 之前定义的 load_arxiv_data
#    如果你重启了内核，请确保把“数据预处理”那个 Cell 先运行一遍，让函数生效。

print("🔄 正在初始化服务，重新构建模型结构...")

# --- B. 加载 ArXiv 模型 ---
arxiv_input, arxiv_linear, arxiv_dnn, _ = load_arxiv_data(ARXIV_CSV, ARXIV_VEC)
service_model_arxiv = DeepFM(arxiv_linear, arxiv_dnn, task='binary', dnn_hidden_units=(256, 128), device=DEVICE)
try:
    service_model_arxiv.load_state_dict(torch.load('deepfm_arxiv_weights.pth', map_location=DEVICE))
    service_model_arxiv.eval()
    print("✅ ArXiv 模型加载成功！")
except FileNotFoundError:
    print("❌ 未找到 ArXiv 权重文件。")


# ==========================================
# 3. 搭建 Flask API
# ==========================================
app = Flask(__name__)

# 定义推荐逻辑 (封装好的工具函数)
def get_recommendation(model, input_data, csv_path, top_k=3):
    # 1. 预测
    pred_ans = model.predict(input_data, batch_size=256)
    # 2. 读原始文件用于展示
    df = pd.read_csv(csv_path)
    df['score'] = pred_ans
    # 3. 排序
    top_items = df.sort_values(by='score', ascending=False).head(top_k)
    # 4. 格式化输出
    results = []
    for _, row in top_items.iterrows():
        results.append({
            "id": str(row['item_id']),
            "title": row['title'],
            "score": float(row['score']),
            "cover": row.get('cover_url', row.get('pdf_url', '')),
            "tags": row.get('tag_names', row.get('category', ''))
        })
    return results

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "device": DEVICE})

@app.route('/recommend', methods=['POST'])
def recommend():
    """
    API 接口
    Input JSON: { "type": "arxiv", "top_k": 3 }
    """
    data = request.json
    rec_type = data.get('type', 'arxiv')
    top_k = data.get('top_k', 3)
    
    try:
        if rec_type == 'arxiv':
            res = get_recommendation(service_model_arxiv, arxiv_input, ARXIV_CSV, top_k)
        else:
            return jsonify({"error": "Unknown type"}), 400
            
        return jsonify({
            "code": 200,
            "type": rec_type,
            "data": res
        })
    except Exception as e:
        return jsonify({"code": 500, "error": str(e)})

# ==========================================
# 4. 启动服务 (在 Notebook 中后台运行)
# ==========================================
def run_app():
    # 端口设置为 6006 或 5000
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# 使用线程启动，避免阻塞 Notebook 的主进程，这样你还能继续跑下面的测试代码
t = threading.Thread(target=run_app)
t.daemon = True
t.start()

print("🚀 服务已启动！监听端口 5000...")
print("你可以运行下一个 Cell 来测试接口。")

## 测试服务 (Client Test)

In [ ]:
# ==========================================
# 🧪 接口测试 (模拟 Dify 调用)
# ==========================================
import requests
import time

# 等待一秒让服务完全启动
time.sleep(2)

url = "http://127.0.0.1:5000/recommend"

# 2. 测试 ArXiv 推荐
print("📡 请求 ArXiv 推荐...")
payload_arxiv = {"type": "arxiv", "top_k": 2}
try:
    resp = requests.post(url, json=payload_arxiv)
    print("ArXiv Response:", resp.json())
except Exception as e:
    print("ArXiv Request Failed:", e)

## 最终服务代码 Cell (直接运行)

In [ ]:
# ==========================================
# 📄 [Cell B] ArXiv DeepFM Server (Port 5001)
# ==========================================
import torch
import pandas as pd
import numpy as np
from deepctr_torch.models import DeepFM
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

# ⚠️⚠️⚠️ 填入你的 Ngrok Token ⚠️⚠️⚠️
NGROK_TOKEN = "这里粘贴你的_Ngrok_Token" 
PORT = 5001 # ArXiv 服务运行在 5001 端口 (防止和 Steam 冲突)

# ================= 1. 加载 ArXiv 模型 =================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
ARXIV_CSV = 'train_arxiv_Researcher_LLM.csv'
ARXIV_VEC = 'arxiv_vectors.npy'

print("🔄 正在初始化 ArXiv 模型...")

try:
    # 假设 load_arxiv_data 已经在前面的 Cell 定义好了
    arxiv_input, arxiv_linear, arxiv_dnn, _ = load_arxiv_data(ARXIV_CSV, ARXIV_VEC)
    
    # 初始化模型结构 (注意 hidden_units 要和训练时一致)
    model_arxiv = DeepFM(arxiv_linear, arxiv_dnn, task='binary', dnn_hidden_units=(256, 128), device=DEVICE)
    
    # 加载权重
    model_arxiv.load_state_dict(torch.load('deepfm_arxiv_weights.pth', map_location=DEVICE))
    model_arxiv.eval()
    print("✅ ArXiv 模型加载成功！Ready to serve.")
except Exception as e:
    print(f"❌ ArXiv 模型加载失败: {e}")

# ================= 2. 定义 Flask 应用 =================
app_arxiv = Flask(__name__)

def get_arxiv_recs(top_k=3):
    # 1. 预测
    pred_scores = model_arxiv.predict(arxiv_input, batch_size=256)
    
    # 2. 读取元数据
    df = pd.read_csv(ARXIV_CSV)
    df['score'] = pred_scores
    
    # 3. 排序
    top_items = df.sort_values(by='score', ascending=False).head(top_k)
    
    # 4. 格式化
    results = []
    for _, row in top_items.iterrows():
        results.append({
            "title": row['title'],
            "score": float(row['score']),
            "type": str(row.get('category', 'Unknown')),
            # 这里取 pdf_url 作为 cover 字段返回，方便前端处理
            "cover": str(row.get('pdf_url', '')) 
        })
    return results

@app_arxiv.route('/recommend', methods=['POST'])
def recommend_arxiv():
    try:
        req = request.json
        top_k = req.get('top_k', 3)
        print(f"📄 [ArXiv] 收到请求, Top {top_k}")
        
        data = get_arxiv_recs(top_k)
        return jsonify({"status": "success", "service": "arxiv", "recommendations": data})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

# ================= 3. 启动服务 =================
def start_arxiv_server():
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill() # 杀掉旧隧道
    
    public_url = ngrok.connect(PORT, bind_tls=True).public_url
    print("\n" + "="*50)
    print(f"📄 ArXiv 服务已启动！API 地址: {public_url}/recommend")
    print("="*50 + "\n")
    
    app_arxiv.run(port=PORT, use_reloader=False)

if __name__ == '__main__':
    t = threading.Thread(target=start_arxiv_server)
    t.daemon = True
    t.start()